# Dataset presentation

## Code imports

In [109]:
import osgeo.gdal as gdal
import osgeo.ogr as ogr
import osgeo.osr as osr
from osgeo.gdalconst import *

# Numpy Import
import numpy as np
import numpy.ma as ma
from skimage import morphology
from sklearn.impute import SimpleImputer, KNNImputer

# Others imports
import os
from datetime import datetime, timedelta
from scipy.ndimage import zoom
import json
import tifffile as tiff
import shutil
import zipfile
from pyproj import Proj, transform
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import shutil
import cv2
import tifffile

## General functions

In [3]:
def get_sentinel_tiles(path_shapefile_sentinel, field):
    """
    Returning an array containing all tilenames in the shapefile tiling.
    @params:
        path_shapefile_sentinel   	- Required  : path to the shapefile containing tiling
        field                      	- Required  : column name in the shapefile to extract tile number
    """
    driver = ogr.GetDriverByName("ESRI Shapefile")
    data_source = driver.Open(path_shapefile_sentinel, 0)
    layer = data_source.GetLayer()

    res = []

    for feature in layer:
        _val = feature.GetField(field)
        res.append(_val)

    return res

In [4]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Unzip a file to a precise directory and remove it after process.
    @params:
        path_to_zip_file   	        - Required  : Path to zip file
        directory_to_extract_to     - Required  : Path to the directory where zip file should be extracted
    """
    if not os.path.exists(directory_to_extract_to):
        os.mkdir(directory_to_extract_to)

    command_unzip = 'unzip -o ' + str(path_to_zip_file) + ' -d ' + str(directory_to_extract_to)
    os.system(command_unzip)
    command_del = 'rm ' + str(path_to_zip_file)
    os.system(command_del)

    print(datetime.now().isoformat() + 'Successfully extracted ' + str(path_to_zip_file))

In [5]:
def merge_two_dicts(x, y):
    """
    Merging two dictionnaries
    @params:
        x   		  - Required  : first dictionnary
        z   		  - Required  : second dictionnary
    """
    z = x.copy()
    z.update(y)
    return z

In [6]:
def extract_dic(tab_classes):
    """
    Transform an array containing classes to a dictionnary
    @params:
        tab_classes   		  - Required  : array containing classes definition
    """
    dic_res = {}

    for e in tab_classes:
        _tmp = e.split(':')
        dic_res[int(_tmp[0])] = int(_tmp[1])

    return dic_res

In [7]:
def resize_bands(dataset_raster):
    """
    Resize bands to correspond at 10m spatial resolution.
    @params:
        dataser_raster   	- Required  : dataset raster
    """
    gt = dataset_raster.GetGeoTransform()
    res_x = int(gt[1])
    res_y = int(abs(gt[-1]))

    _res_band = None
    if res_x != 10 or res_y != 10:
        ratio = float(res_x / 10)
        to_resample = dataset_raster.GetRasterBand(1).ReadAsArray()
        _res_band = np.array(zoom(to_resample, ratio, order=0))
    else:
        _res_band = np.array(dataset_raster.GetRasterBand(1).ReadAsArray())

    return _res_band

In [8]:
def list_to_string(list, sep=';'):
    """
    Convert a list to string using a separator between each element.
    @params:
        list   	- Required  : list to convert
        sep     - Required  : separator to put between each element
    """
    res = None

    for e in list:
        if res is None:
            res = str(e)
        else:
            res += sep + str(e)

    return res

## Sentinel-2 processing

In [9]:
def download_sentinel2(tile, start_date, end_date):
    """
    Download every Sentinel-2 L2A images between two dates.
    @params:
        tile   	                    - Required  : tilename
        start_date                  - Required  : start date
        end_date                    - Required  : end date
    """
    command = "python ./sentinel_download/theia_download.py -t T" + str(
        tile) + " -c SENTINEL2 -a ./sentinel_download/config_theia.cfg + -d " + str(start_date) + " -f " + str(
        end_date) + " -m 10"
    os.system(command)

## OCSGE2 processing

In [10]:
def change_projection(path_raster_reference, path_shapefile_to_raster, attribute_shapefile_n5, attribute_shapefile_n4, output_filename,
                      attribute_values_n5=None, attribute_values_n4=None):
    """
    Changing shapefile projection to match with raster.
    @params:
        path_raster_reference   	- Required  : raster or satellite image as reference
        path_shapefile_to_raster   	- Required  : shapefile to rasterize
        attribute_shapefile   		- Required  : attribute to rasterize (name of shapefile column)
        output_filename   			- Required  : output file rasterized
        attribute_values   			- Required  : list of attributes to rasterize
    """
    if attribute_values_n4 is None:
        attribute_values_n4 = []
    if attribute_values_n5 is None:
        attribute_values_n5 = []
    if (len(attribute_values_n5) == 0):
        raise Exception('Error : length of attributes array is 0')

    tif = gdal.Open(path_raster_reference)
    driver = ogr.GetDriverByName("ESRI Shapefile")

    dataSource = driver.Open(path_shapefile_to_raster, 0)

    layer = dataSource.GetLayer()

    sourceprj = layer.GetSpatialRef()
    targetprj = osr.SpatialReference(wkt=tif.GetProjection())
    transform = osr.CoordinateTransformation(sourceprj, targetprj)

    to_fill = ogr.GetDriverByName("ESRI Shapefile")
    ds = to_fill.CreateDataSource(output_filename)
    outlayer = ds.CreateLayer('', targetprj, ogr.wkbPolygon)
    outlayer.CreateField(ogr.FieldDefn('ID', ogr.OFTInteger))
    outlayer.CreateField(ogr.FieldDefn(attribute_shapefile_n5, ogr.OFTInteger))

    i = 0

    for feature in layer:
        _val = None

        if (feature.GetField(attribute_shapefile_n5) is not None) and (feature.GetField(attribute_shapefile_n5) != 0) :
            if int(feature.GetField(attribute_shapefile_n5)) in attribute_values_n5:
                _val = feature.GetField(attribute_shapefile_n5)
            else:
                _val = 0
            transformed = feature.GetGeometryRef()
            transformed.Transform(transform)

            geom = ogr.CreateGeometryFromWkb(transformed.ExportToWkb())
            defn = outlayer.GetLayerDefn()
            feat = ogr.Feature(defn)
            feat.SetField('ID', i)
            feat.SetField(attribute_shapefile_n5, _val)
            feat.SetGeometry(geom)
            outlayer.CreateFeature(feat)
            i += 1
            feat = None
        else:
            if int(feature.GetField(attribute_shapefile_n4)) in attribute_values_n4:
                _val = feature.GetField(attribute_shapefile_n4)
            else:
                _val = 0
            transformed = feature.GetGeometryRef()
            transformed.Transform(transform)

            geom = ogr.CreateGeometryFromWkb(transformed.ExportToWkb())
            defn = outlayer.GetLayerDefn()
            feat = ogr.Feature(defn)
            feat.SetField('ID', i)
            feat.SetField(attribute_shapefile_n5, _val)
            feat.SetGeometry(geom)
            outlayer.CreateFeature(feat)
            i += 1
            feat = None

    ds = None

In [11]:
def rasterize_ground_ref(path_raster_reference, path_shapefile_to_raster, attribute_shapefile, output_filename):
    """
    Rasterizing ground truth.
    @params:
        path_raster_reference   	- Required  : raster or satellite image as reference
        path_shapefile_to_raster   	- Required  : shapefile to rasterize
        attribute_shapefile   		- Required  : attribute to rasterize (name of shapefile column)
        output_filename   			- Required  : output file rasterized
    """
    dataset_raster = gdal.Open(path_raster_reference, GA_ReadOnly)
    geo_transform = dataset_raster.GetGeoTransform()

    dataset_shapefile = ogr.Open(path_shapefile_to_raster)
    layer_shapefile = dataset_shapefile.GetLayer()

    nodata_value = 255
    # source_layer = data.GetLayer()

    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * dataset_raster.RasterXSize
    y_min = y_max + geo_transform[5] * dataset_raster.RasterYSize
    x_res = dataset_raster.RasterXSize
    y_res = dataset_raster.RasterYSize
    pixel_width = geo_transform[1]

    target_ds = gdal.GetDriverByName('GTiff').Create(output_filename, x_res, y_res, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(dataset_raster.GetProjection())
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(nodata_value)
    band.FlushCache()
    band.Fill(nodata_value)

    gdal.RasterizeLayer(target_ds, [1], layer_shapefile, options=["ATTRIBUTE=" + str(attribute_shapefile)])

    target_ds = None

In [60]:
def merge_classes(input_raster, dic_classes, output_filename):
    """
    Merging some classes togerther to get a new typology
    @params:
        input_raster   	- Required  : raster to merge
        dic_classes   	- Required  : dictionnary containing original classes as keys, and new typologies as value
        output_filename - Required  : output path to create the new raster
    """
    ds_raster = gdal.Open(input_raster, GA_ReadOnly)
    array_raster = ds_raster.GetRasterBand(1).ReadAsArray()
    tab_return = None

    for key in dic_classes:
        if tab_return is None:
            tab_return = np.where(array_raster == key, dic_classes[key], 0)
        else:
            tab_return += np.where(array_raster == key, dic_classes[key], 0)

    if -99 in tab_return:
        tab_return = ma.masked_array(tab_return, tab_return == -99)
        for shift in (-3, 3):
            for axis in (0, 1):
                a_shifted = np.roll(tab_return, shift=shift, axis=axis)
                idx = ~a_shifted.mask * tab_return.mask
                tab_return[idx] = a_shifted[idx]

    '''se_disk = morphology.disk(3)
    se_rectangle = morphology.rectangle(3, 3)
    tab_return = morphology.opening(tab_return, se_disk)
    tab_return = morphology.opening(tab_return, se_rectangle)'''

    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(output_filename, ds_raster.RasterXSize, ds_raster.RasterYSize, 1, gdal.GDT_Byte)
    dst_ds.SetProjection(ds_raster.GetProjection())
    geotransform = ds_raster.GetGeoTransform()
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.GetRasterBand(1).WriteArray(tab_return)
    dst_ds = None

In [13]:
def computing_roads(output_folder, shapefile_roads, roads_pix_value, output_merging_tif, output_name_with_roads):
    """
    Adding roads to the ground reference rasters
    @params:
        output_folder   	- Required  : raster to merge
        shapefile_roads   	- Required  : dictionnary containing original classes as keys, and new typologies as value
        roads_pix_value     - Required  : output path to create the new raster
        output_merging_tif  - Required  : output path to create the new raster
        output_name_with_roads  - Required  : output path to create the new raster
    """
    output_filename_reproj = os.path.join(output_folder, 'roads_reproj.shp')
    change_projection(output_merging_tif, shapefile_roads, 'VAL', None, output_filename_reproj, [99], None)

    output_filename_rasterize = os.path.join(output_folder, 'roads_reproj.tif')
    rasterize_ground_ref(output_merging_tif, output_filename_reproj, 'VAL', output_filename_rasterize)

    dataset_roads = gdal.Open(output_filename_rasterize, GA_ReadOnly)
    dataset_gt = gdal.Open(output_merging_tif, GA_ReadOnly)

    band_roads = dataset_roads.GetRasterBand(1).ReadAsArray()
    band_gt = dataset_gt.GetRasterBand(1).ReadAsArray()

    band_roads[band_roads == 255] = 0
    _tmp = band_gt + band_roads

    _tmp[_tmp >= 90] = roads_pix_value
    _tmp[_tmp == 25] = 5

    shape = _tmp.shape

    output = output_name_with_roads

    geo_transform = dataset_gt.GetGeoTransform()
    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(output, shape[1], shape[0], 1, gdal.GDT_UInt16)
    dst_ds.SetProjection(dataset_gt.GetProjection())
    dst_ds.SetGeoTransform(geo_transform)
    dst_ds.GetRasterBand(1).WriteArray(_tmp)

    return output

In [120]:
def connected_components(path_ground_reference, output_filename, min_size=400):
    dataset_gt = gdal.Open(path_ground_reference, GA_ReadOnly)
    gr = dataset_gt.GetRasterBand(1).ReadAsArray()
    #gr = gr[0:1000,0:1000]
    
    max_gr = np.amax(gr)
    res = None
    mask = np.uint8(np.where(gr == 0, 255, 0))
    
    for i in range(1, max_gr + 1):
        tmp_array_1 = np.uint8(np.where(gr == i, 127, 255))
        tmp_array_1 = cv2.threshold(tmp_array_1, 127, 255, cv2.THRESH_BINARY)[1]
        
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(tmp_array_1, connectivity=8)
        sizes = stats[1:, -1]; nb_components = nb_components - 1
        img1 = np.zeros((output.shape), dtype=int)

        for j in range(0, nb_components):
            if sizes[j] >= min_size:
                img1[output == j + 1] = 255
        
        img1 = np.uint8(np.where(img1 == 0, 255, 127))
        tmp_array_2 = cv2.threshold(img1, 127, 255, cv2.THRESH_BINARY)[1]
                
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(tmp_array_2, connectivity=8)
        sizes = stats[1:, -1]; nb_components = nb_components - 1
        img2 = np.zeros((output.shape), dtype=int)

        for k in range(0, nb_components):
            if sizes[k] >= min_size:
                img2[output == k + 1] = 255
        
        if res is None:
            res = np.zeros((img2.shape), dtype=int)
        
        img2 = np.where(img2 == 0, 0, i)
    
        '''geo_transform = dataset_gt.GetGeoTransform()
        driver = gdal.GetDriverByName("GTiff")
        dst_ds = driver.Create('/media/wenger/LaCie/create_dataset/tmp/' + str(i) + '_img2.tif', res.shape[1], res.shape[0], 1, gdal.GDT_UInt16, options=["COMPRESS=LZW"])
        dst_ds.SetProjection(dataset_gt.GetProjection())
        dst_ds.SetGeoTransform(geo_transform)
        dst_ds.GetRasterBand(1).WriteArray(img2)'''
        
        res += img2
        
        '''mask_res_tmp = np.where(img1 != 0, 1, 0)
        res *= mask_res_tmp
        res += np.where(img1 == 0, i, 0)
        res += mask
        res = np.where(res >= 255, 255, res)'''
    
    res += mask
    res = np.uint8(np.where(res >= 255, 255, res))
    
    #imputer = SimpleImputer(missing_values=0, strategy='mean')
    imputer = KNNImputer(missing_values=0, n_neighbors=1)
    #to_mask = np.uint8(np.where(res == 0, np.nan, res))
    dst = imputer.fit_transform(res)
    
    geo_transform = dataset_gt.GetGeoTransform()
    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(output_filename, res.shape[1], res.shape[0], 1, gdal.GDT_UInt16, options=["COMPRESS=LZW"])
    dst_ds.SetProjection(dataset_gt.GetProjection())
    dst_ds.SetGeoTransform(geo_transform)
    dst_ds.GetRasterBand(1).WriteArray(dst)

In [ ]:
path_gr = '/media/wenger/LaCie/create_dataset/ground_reference_raster/32ULU_ground_reference.tif'
tmp = '/media/wenger/LaCie/create_dataset/tmp/32ULU_test.tif'
connected_components(path_gr, tmp)

## Sentinel-1 processing

In [14]:
def download_sentinel1(tiles_list, directory_images, step=8):
    """
    Download every Sentinel-2 L2A images between two dates.
    @params:
        tiles_list   	        - Required  : list of Sentinel-2 tiles
        directory_images        - Required  : directory containing Sentinel-2 images
        step                    - Optional  : day interval between Sentinel-2 image to download Sentinel-1 images
    """
    s1_output = './S1_img'
    for tile in tiles_list:
        s1_tile = tile

        for img in os.listdir(directory_images):
            if s1_tile in img:
                _date = get_date_from_image_path(img)
                datetime_object = datetime.strptime(_date, '%Y%m%d')
                s1_first_date = (datetime_object - timedelta(days=int(step))).strftime('%Y-%m-%d')
                s1_last_date = (datetime_object + timedelta(days=int(step))).strftime('%Y-%m-%d')

                replacements = {'{output_path}': s1_output, '{first_date}': s1_first_date, '{last_date}': s1_last_date,
                                '{tiles}': s1_tile}

                with open('./sentinel_1_tiling/S1Processor.cfg') as infile, open(
                        './sentinel_1_tiling/S1Processor_tmp.cfg', 'w') as outfile:
                    for line in infile:
                        for src, target in replacements.iteritems():
                            line = line.replace(src, target)
                        outfile.write(line)

                command = 'python ./sentinel_1_tiling/S1Processor.py ./sentinel_1_tiling/S1Processor_tmp.cfg'
                os.system(command)
                os.system('rm ./sentinel_1_tiling/S1Processor_tmp.cfg')

In [15]:
def check_satellite_data(tiles, directory_s2, directory_s1, step=8, filtered=True):
    """
    Construct JSON file to create database of S1 and S2 images.
    @params:
        tiles   	        - Required  : list of Sentinel-2 tiles
        directory_s2        - Required  : directory containing Sentinel-2 images
        directory_s1        - Required  : directory containing Sentinel-1 images pretreated from s1tiling
        step                - Optional  : day interval between Sentinel-2 image to download Sentinel-1 images
        filtered            - Optional  : using or not filtered images
    """
    data = {}

    for tile in tiles:
        data[tile] = []
        for img_s2 in os.listdir(directory_s2):
            if tile in img_s2:
                date = get_date_from_image_path(img_s2)
                s1_path_vv = get_s1_from_s2_tile(tile, date, directory_s1, 'vv', step=step, filtered=filtered)
                s1_path_vh = get_s1_from_s2_tile(tile, date, directory_s1, 'vh', step=step, filtered=filtered)
                data[tile].append({
                    'dateS2': date,
                    's2_path': os.path.join(directory_s2, img_s2),
                    's1_path_vv': s1_path_vv,
                    's1_path_vh': s1_path_vh
                })

    if os.path.exists('./sentinel_data.json'):
        os.remove('./sentinel_data.json')

    with open('./sentinel_data.json', 'w') as file:
        json.dump(data, file)

    return data

In [16]:
def get_s1_from_s2_tile(tile, date_s2, directory_s1, pol, step=8, filtered=True):
    """
    Extract filename of the Sentinel-1 image
    @params:
        tile   	            - Required  : current tile
        date_s2             - Required  : date of current Sentinel-2
        directory_s1        - Required  : directory containing Sentinel-1 images pretreated from s1tiling
        step                - Optional  : day interval between Sentinel-2 image to download Sentinel-1 images
        filtered            - Optional  : using or not filtered images
    """
    path_folder = os.path.join(directory_s1, str(tile))
    if filtered:
        path_folder = os.path.join(directory_s1, str(tile), 'filtered')

    s1_files = []
    datetime_s2 = datetime.strptime(date_s2, '%Y%m%d')
    s1_first_date = datetime_s2 - timedelta(days=int(step))
    s1_last_date = datetime_s2 + timedelta(days=int(step))

    for file in os.listdir(path_folder):
        if (tile in file) and (pol in file):
            datetime_current_s1 = datetime.strptime(get_date_from_image_path(file, s1=True), '%Y%m%d')
            if s1_first_date <= datetime_current_s1 <= s1_last_date:
                s1_files.append(file)

    _size = None
    res_img = None
    for file in s1_files:
        _img = tiff.imread(os.path.join(path_folder, file))
        if _size is None:
            _size = np.count_nonzero(_img == 0)
            res_img = file
        else:
            if np.count_nonzero(_img == 0) < _size:
                res_img = file

    return os.path.join(path_folder, res_img)

In [17]:
def get_date_from_image_path(name, s1=False):
    """
    Return date from Sentinel-2 folder name or Sentinel-1 pretreated name.
    @params:
        path   	                    - Required  : Folder name containing every image bands.
        s1   	                    - Optional  : True if name is Sentinel-1 filename
    """
    if s1:
        date = name.split('_')[5][0:8]
    else:
        date = name.split('_')[1].split('-')[0]

    return date

## Generate Dataset

In [18]:
def create_folders_dataset(path_directory_dataset):
    if os.path.exists(os.path.join(path_directory_dataset, 's1')):
        shutil.rmtree(os.path.join(path_directory_dataset, 's1'))
        os.mkdir(os.path.join(path_directory_dataset, 's1'))
    else:
        os.mkdir(os.path.join(path_directory_dataset, 's1'))

    if os.path.exists(os.path.join(path_directory_dataset, 's2')):
        shutil.rmtree(os.path.join(path_directory_dataset, 's2'))
        os.mkdir(os.path.join(path_directory_dataset, 's2'))
    else:
        os.mkdir(os.path.join(path_directory_dataset, 's2'))

    if os.path.exists(os.path.join(path_directory_dataset, 'ground_reference')):
        shutil.rmtree(os.path.join(path_directory_dataset, 'ground_reference'))
        os.mkdir(os.path.join(path_directory_dataset, 'ground_reference'))
    else:
        os.mkdir(os.path.join(path_directory_dataset, 'ground_reference'))

    if os.path.exists(os.path.join(path_directory_dataset, 'labels')):
        shutil.rmtree(os.path.join(path_directory_dataset, 'labels'))
        os.mkdir(os.path.join(path_directory_dataset, 'labels'))
    else:
        os.mkdir(os.path.join(path_directory_dataset, 'labels'))

In [19]:
def calculate_patches_tile(tile, directory_ground_reference, directory_sat_images_s2, directory_sat_images_s1,
                           directory_dataset, patch_size, method=1):
    bands_S2 = ['B2.tif', 'B3.tif', 'B4.tif', 'B5.tif', 'B6.tif', 'B7.tif', 'B8.tif', 'B8A.tif', 'B11.tif', 'B12.tif']
    polar_S1 = ['vv', 'vh']
    path_gr = None

    for gr in os.listdir(directory_ground_reference):
        if tile in gr:
            path_gr = os.path.join(directory_ground_reference, gr)

    assert path_gr is not None

    ds_gr_raster = gdal.Open(path_gr)
    gr_raster = ds_gr_raster.GetRasterBand(1).ReadAsArray()

    if method == 1:
        step = patch_size
    else:
        step = 1

    for y in range(0, gr_raster.shape[0], step):
        for x in range(0, gr_raster.shape[1], step):
            if method == 2:
                step = 1

            gr_patch = gr_raster[y:y + patch_size, x:x + patch_size]

            if 0 not in gr_patch:
                if method == 2:
                    step = patch_size + 1

                x_coords_topleft, y_coords_topleft, x_size, y_size = get_coordinates(ds_gr_raster,
                                                                                     x, y)
                new_geotrans = (x_coords_topleft, x_size, 0, y_coords_topleft, 0, y_size)

                driver = gdal.GetDriverByName("GTiff")

                for img_dir in os.listdir(directory_sat_images_s2):
                    if tile in img_dir:
                        date_s2 = get_date_from_image_path(img_dir)
                        output_patch_name_s2 = os.path.join(directory_dataset, 's2', str(tile) + '_' + str(date_s2) + '_S2_' +
                                                            str(x) + '_' + str(y) + '.tif')
                        output_patch_name_gr = os.path.join(directory_dataset, 'ground_reference', str(tile) + '_GR_' +
                                                            str(x) + '_' + str(y) + '.tif')

                        dst_ds_img_s2 = driver.Create(output_patch_name_s2, patch_size, patch_size, len(bands_S2),
                                                      gdal.GDT_Float32, options=["COMPRESS=LZW"])
                        dst_ds_img_s2.SetProjection(ds_gr_raster.GetProjection())
                        dst_ds_img_s2.SetGeoTransform(new_geotrans)

                        if not os.path.exists(output_patch_name_gr):
                            dst_ds_mask = driver.Create(output_patch_name_gr, patch_size, patch_size, 1, gdal.GDT_UInt16,
                                                    options=["COMPRESS=LZW"])
                            dst_ds_mask.SetProjection(ds_gr_raster.GetProjection())
                            dst_ds_mask.SetGeoTransform(new_geotrans)
                            dst_ds_mask.GetRasterBand(1).WriteArray(gr_patch)

                        count_s2_bands = 1
                        for band_name in bands_S2:
                            for band in os.listdir(os.path.join(directory_sat_images_s2, img_dir)):
                                if ('SRE' in band) and (band_name in band):
                                    ds_band = gdal.Open(os.path.join(directory_sat_images_s2, img_dir, band))
                                    patch_band = resize_bands(ds_band)[y:y + patch_size, x:x + patch_size]
                                    dst_ds_img_s2.GetRasterBand(count_s2_bands).WriteArray(patch_band)
                                    count_s2_bands += 1

                for folder_tile in os.listdir(directory_sat_images_s1):
                    if str(folder_tile) == str(tile):
                        for year in os.listdir(os.path.join(directory_sat_images_s1, folder_tile)):
                            for month in os.listdir(os.path.join(directory_sat_images_s1, folder_tile, year)):
                                complete_dates_s1 = get_s1_dates(os.path.join(directory_sat_images_s1, folder_tile,
                                                                              year, month))

                                for date in complete_dates_s1:
                                    count_s1_polar = 1
                                    date_s1 = str(year) + str(month) + str(date[-2:])
                                    output_patch_name_s1 = os.path.join(directory_dataset, 's1',
                                                                        str(tile) + '_' + str(date_s1) + '_S1_' +
                                                                        str(x) + '_' + str(y) + '.tif')
                                    dst_ds_img_s1 = driver.Create(output_patch_name_s1, patch_size, patch_size,
                                                                  len(polar_S1),
                                                                  gdal.GDT_Float32, options=["COMPRESS=LZW"])
                                    dst_ds_img_s1.SetProjection(ds_gr_raster.GetProjection())
                                    dst_ds_img_s1.SetGeoTransform(new_geotrans)

                                    for polar in polar_S1:
                                        for img in os.listdir(
                                                os.path.join(directory_sat_images_s1, folder_tile, year, month)):
                                            if (polar in img) and (date in img):
                                                path_radar = os.path.join(directory_sat_images_s1, folder_tile, year,
                                                                          month, img)
                                                ds_band = gdal.Open(path_radar)
                                                patch_band = resize_bands(ds_band)[y:y + patch_size, x:x + patch_size]
                                                dst_ds_img_s1.GetRasterBand(count_s1_polar).WriteArray(patch_band)
                                                count_s1_polar += 1

In [93]:
def create_json_labels(path_dataset):
    """
    Construct JSON file to generate labels for scene classification.
    @params:
        path_ground_reference   	- Required  : path to ground reference folder
        path_output_json            - Required  : path to the json directory
    """
    for file in os.listdir(os.path.join(path_dataset, 'ground_reference')):
        data = {}
        patch_infos = extract_info_patches(os.path.basename(file), is_gr=True)
        data['corresponding_s2'] = get_corresponding_sat(os.path.join(path_dataset, 's2'), patch_infos[0], patch_infos[1],
                                                         patch_infos[2])
        data['corresponding_s1'] = get_corresponding_sat(os.path.join(path_dataset, 's1'), patch_infos[0], patch_infos[1],
                                                         patch_infos[2])
        data['projection'] = get_projection_patch(os.path.join(path_dataset, 'ground_reference', file))
        data['labels'] = list_to_string(extract_classes_gr(os.path.join(path_dataset, 'ground_reference', file)).keys())

        json_name = patch_infos[0] + '_' + patch_infos[1] + '_' + patch_infos[2] + '.json'

        if os.path.exists(os.path.join(path_dataset, 'labels', json_name)):
            os.remove(os.path.join(path_dataset, 'labels', json_name))

        with open(os.path.join(path_dataset, 'labels', json_name), 'w') as file:
            json.dump(data, file)

In [86]:
def extract_classes_gr(path_gr_patch):
    """
    Extract classes in ground reference patch.
    @params:
        path_gr_patch   	- Required  : filename of the patch
    """
    dataset = gdal.Open(path_gr_patch, GA_ReadOnly)
    array = dataset.GetRasterBand(1).ReadAsArray()

    unique, counts = np.unique(array, return_counts=True)

    return dict(zip(unique, counts))

In [84]:
def get_corresponding_sat(path_folder, tile, tile_x, tile_y):
    """
    Extract every file in the folder corresponding to the coordinates and concatenate them in one single string.
    @params:
        path_folder   	- Required  : filename of the patch
        tile_x          - Required  : boolean to tell if the filename is ground reference or not
        tile_y          - Required  : boolean to tell if the filename is ground reference or not
    """
    output = None

    for file in os.listdir(path_folder):
        string = tile_x + '_' + tile_y
        if (string in file) and (tile in file):
            if output is None:
                output = file
            else:
                output += ';' + file

    return output

In [87]:
def get_s1_dates(path):
    res = []

    assert len(os.listdir(path)) > 2

    for img in os.listdir(path):
        _c = img.split('_')[5][0:8]
        if _c not in res:
            res.append(_c)

    return res

In [92]:
def extract_info_patches(patch_name, is_gr=False):
    """
    Extract informations from patch name.
    @params:
        patch_name   	- Required  : filename of the patch
        is_gr           - Required  : boolean to tell if the filename is ground reference or not
    """
    split = patch_name.split('_')

    if is_gr:
        tile = split[0]
        tile_x = split[2]
        tile_y = split[3].split('.')[0]
        output = (tile, tile_x, tile_y)
    else:
        tile = split[0]
        date_img = split[1]
        sat = split[2]
        tile_x = split[3]
        tile_y = split[4]
        output = (tile, date_img, sat, tile_x, tile_y)

    return output

In [25]:
def get_coordinates(dataset, x_index_topleft, y_index_topleft):
    """
    Get upper left coordinates using matrix cell number
    @params:
        dataset   		  - Required  : dataset raster
        x_index_topleft   - Required  : x cell index
        y_index_topleft   - Required  : y cell index
    """
    if dataset is not None:
        (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = dataset.GetGeoTransform()

        x_coords_topleft = x_index_topleft * x_size + upper_left_x
        y_coords_topleft = y_index_topleft * y_size + upper_left_y

        return (x_coords_topleft, y_coords_topleft, x_size, y_size)
    else:
        raise Exception('Error : dataset is None')

In [26]:
def get_projection_patch(path_patch):
    """
    Get the projection (WKT) of a patch.
    @params:
        path_patch   	- Required  : path to the patch
    """
    dataset = gdal.Open(path_patch, GA_ReadOnly)
    proj = dataset.GetProjection()

    return proj

## Checking overlap between patches

In [231]:
class Patch:
    x = 0
    y = 0
    tile = None
    
    def __init__(self, x, y, tile):
        self.x = x
        self.y = y
        self.tile = tile
    
    def affiche(self):
        print("x =", self.x, "y =", self.y, "tile =", self.tile)

In [230]:
def delete_overlapping_patches(directory_dataset):
    """
    Finding an overlap between two rasters. This function works even if rasters have different projection.
    @params:
        directory_dataset   	- Required  : path to the first raster
    """
    path_gr = os.path.join(directory_dataset, 'ground_reference')
    path_s1 = os.path.join(directory_dataset, 's1')
    path_s2 = os.path.join(directory_dataset, 's2')
    path_labels = os.path.join(directory_dataset, 'labels')
    
    dic_res = []
    
    if os.path.exists(os.path.join(directory_dataset, 'overlapping.txt')):
        os.remove(os.path.join(directory_dataset, 'overlapping.txt'))
    
    for patch1 in os.listdir(path_gr):
        tile_patch1 = patch1.split('_')[0]
        for patch2 in os.listdir(path_gr):
            tile_patch2 = patch2.split('_')[0]
            if tile_patch1 != tile_patch2:
                if find_raster_intersect(os.path.join(path_gr, patch1), os.path.join(path_gr, patch2)):
                    x = patch1.split('_')[2]
                    y = patch1.split('_')[3].split('.')[0]

                    patch_to_delete = Patch(x, y, tile_patch1)
                    dic_res.append(patch_to_delete)
                    
                    with open(os.path.join(directory_dataset, 'overlapping.txt'), "a") as f:
                        f.write(str(patch1) + ' overlap ' + str(patch2) + '\n')
    
    for patch in dic_res:
        delete_every_patch(patch.x, patch.y, patch.tile, directory_dataset)

In [229]:
def delete_every_patch(x, y, tile, directory_dataset):
    """Delete every patch matching x, y and tile
    @params:
        x   	- Required  : x coordinates of the patch
        y   	- Required  : y coordinates of the patch
        tile   	- Required  : tile of the patch
        directory_dataset   	- Required  : dataset directory
    """
    path_gr = os.path.join(directory_dataset, 'ground_reference')
    path_s1 = os.path.join(directory_dataset, 's1')
    path_s2 = os.path.join(directory_dataset, 's2')
    path_labels = os.path.join(directory_dataset, 'labels')
    
    #Delete groud reference patch
    for f in os.listdir(os.path.join(directory_dataset, path_gr)):
        tile_f = f.split('_')[0]
        x_f = f.split('_')[2]
        y_f = f.split('_')[3].split('.')[0]
        
        if (str(x) == str(x_f)) and (str(y_f) == str(y)) and (str(tile) == str(tile_f)):
            os.remove(os.path.join(directory_dataset, path_gr, f))
            
    #Delete S1 patch
    for f in os.listdir(os.path.join(directory_dataset, path_s1)):
        tile_f = f.split('_')[0]
        x_f = f.split('_')[3]
        y_f = f.split('_')[4].split('.')[0]
        
        if (str(x) == str(x_f)) and (str(y_f) == str(y)) and (str(tile) == str(tile_f)):
            os.remove(os.path.join(directory_dataset, path_s1, f))
    
    #Delete S2 patch
    for f in os.listdir(os.path.join(directory_dataset, path_s2)):
        tile_f = f.split('_')[0]
        x_f = f.split('_')[3]
        y_f = f.split('_')[4].split('.')[0]
        
        if (str(x) == str(x_f)) and (str(y_f) == str(y)) and (str(tile) == str(tile_f)):
            os.remove(os.path.join(directory_dataset, path_s2, f))
            
    #Delete labels patch
    for f in os.listdir(os.path.join(directory_dataset, path_labels)):
        tile_f = f.split('_')[0]
        x_f = f.split('_')[1]
        y_f = f.split('_')[2].split('.')[0]
        
        if (str(x) == str(x_f)) and (str(y_f) == str(y)) and (str(tile) == str(tile_f)):
            os.remove(os.path.join(directory_dataset, path_labels, f))
        

In [228]:
def find_raster_intersect(path_raster1, path_raster2, with_text=False):
    """
    Finding an overlap between two rasters. This function works even if rasters have different projection.
    @params:
        path_raster1   	- Required  : path to the first raster
        path_raster2   	- Required  : path to the second raster
    """
    overlap = False
    
    raster1 = gdal.Open(path_raster1)
    raster2 = gdal.Open(path_raster2)
    gt1 = raster1.GetGeoTransform()
    gt2 = raster2.GetGeoTransform()
    
    proj_raster1 = osr.SpatialReference(wkt=raster1.GetProjection())
    epsg_raster1 = proj_raster1.GetAttrValue('AUTHORITY',1)
    proj_raster2 = osr.SpatialReference(wkt=raster2.GetProjection())
    epsg_raster2 = proj_raster2.GetAttrValue('AUTHORITY',1)

    # find each image's bounding box
    r1 = [gt1[0], gt1[3], gt1[0] + (gt1[1] * raster1.RasterXSize), gt1[3] + (gt1[5] * raster1.RasterYSize)]
    r1_x1, r1_y1 = change_coordinates(r1[0], r1[1], epsg_raster1)
    r1_x2, r1_y2 = change_coordinates(r1[2], r1[3], epsg_raster1)
    r1 = [r1_x1, r1_y1, r1_x2, r1_y2]
    
    r2 = [gt2[0], gt2[3], gt2[0] + (gt2[1] * raster2.RasterXSize), gt2[3] + (gt2[5] * raster2.RasterYSize)]
    r2_x1, r2_y1 = change_coordinates(r2[0], r2[1], epsg_raster2)
    r2_x2, r2_y2 = change_coordinates(r2[2], r2[3], epsg_raster2)
    r2 = [r2_x1, r2_y1, r2_x2, r2_y2]
    
    if with_text:
        print('\t1 bounding box: %s' % str(r1))
        print('\t2 bounding box: %s' % str(r2))

    # find intersection between bounding boxes
    intersection = [max(r1[0], r2[0]), min(r1[1], r2[1]), min(r1[2], r2[2]), max(r1[3], r2[3])]
    if r1 != r2:
        # check for any overlap at all...
        if (intersection[2] < intersection[0]) or (intersection[1] < intersection[3]):
            overlap = False
            if with_text:
                print('\tThat\'s not an overlap !')
        else:
            overlap = True
            if with_text:
                print('\tThat\'s an overlap !')

    return overlap

In [227]:
def change_coordinates(x, y, epsg_from, epsg_to='4326'):
    """
    Changing coordinates from an epsg to an other.
    @params:
        x   	        - Required  : x coordinate
        y   	        - Required  : y coordinate
        epsg_from   	- Required  : epsg of x and y coordinates
        epsg_to   	    - Required  : epsg where they need to be converted
    """
    inProj = Proj(init='epsg:' + epsg_from)
    outProj = Proj(init='epsg:' + epsg_to)
    x2,y2 = transform(inProj,outProj,x,y)
    
    return x2, y2

In [226]:
def delete_zero_patch(directory_dataset):
    """
    Deleting patches where ground reference contains zeros.
    @params:
        directory_dataset   	- Required  : path to the first raster
    """
    path_gr = os.path.join(directory_dataset, 'ground_reference')
    path_s1 = os.path.join(directory_dataset, 's1')
    path_s2 = os.path.join(directory_dataset, 's2')
    path_labels = os.path.join(directory_dataset, 'labels')
    
    for patch in os.listdir(path_gr):
        raster = gdal.Open(os.path.join(path_gr, patch))
        array = raster.GetRasterBand(1).ReadAsArray()
        
        if 0 in array:
            tile = patch.split('_')[0]
            x = patch.split('_')[2]
            y = patch.split('_')[3].split('.')[0]
            delete_every_patch(x, y, tile, directory_dataset)

## Runs

This part regroups all the codes enabling the preprocessing of the data and the creation of the dataset. It is organized in the same way as the previous functions with a sub-part per treatment.

### Variables

In [2]:
cwd = os.getcwd()
# TODO: Regroupement des classes
urban_regroup_classes = '11111:1 11112:1 11113:1 11121:1 11122:1 11123:1 11211:2 11212:2 11213:2 11221:2 11222:2 11223:2 11231:2 11232:2 11233:2 11241:2 11242:2 11243:2 11301:2 11302:2 11303:2 11401:3 11402:3 11403:3 12111:3 12112:3 12113:3 12121:3 12122:3 12123:3 12131:3 12132:3 12133:4 13141:3 13142:3 13123:4 12151:3 12152:3 12153:4 12201:3 12202:3 12203:4 13111:3 13112:3 13113:3 13121:3 13122:3 13123:3 13131:3 13132:3 13133:3 13141:3 13142:3 13143:4 13201:1 13202:3 13203:3 13301:2 13302:2 13303:2 13401:3 13402:3 13403:3 14111:3 14112:3 14113:5 14201:3 14202:3 14203:4 14301:3 14302:3 14303:3 15101:3 15102:3 15103:4 16101:3 16102:3 16103:3 17101:1 17102:1 17103:1 12141:4 12142:4 12143:3 14131:5 14132:5 14133:5 14121:-99 14122:-99 14123:-99'
natural_regroup_classes = '2110:6 2120:6 2210:7 2221:8 2222:8 2223:8 2310:9 2320:10 3110:11 3120:11 3130:11 3140:11 3150:11 3210:12 3220:12 3230:12 3310:13 3320:13 3340:13 4110:14 4120:14 5110:15 5120:15 5130:15'
path_shapefile_sentinel = './inputs/S2_Tiling_GE_test.shp'
field = 'Name'
directory_images_s2 = os.path.join(cwd, 'S2_img')
directory_images_s1_not_compressed = '/media/wenger/LaCie/S1_img'
directory_images_s1_compressed = '/media/wenger/DATA2/S1_img'
shapefile_roads = './inputs/routes_finales_GE.shp'
patch_size = 256
directory_ground_reference_raster = os.path.join(cwd, 'ground_reference_raster')
directory_tmp = os.path.join(cwd, 'tmp')
directory_dataset = os.path.join(cwd, 'dataset')
path_shp_ground_reference = './inputs/merge_grand_est.shp'
attribute_shp_ground_reference_urban = 'cod_n5'
attribute_shp_ground_reference_natural = 'cod_n4'

# Read all S2-Tiles and put them in an array
# tiles_list = get_sentinel_tiles(path_shapefile_sentinel, field)
#tiles_list = ['31UGP', '32ULU', '32ULV', '32UMU', '32UMV', '31UGQ', '31UFQ', '31UFP', '32TLT']
tiles_list = ['32ULU']

# Download Sentinel for each tile
start_date = '2020-10-01'
end_date = '2020-12-31'

### S2

In [47]:
print(datetime.now().isoformat() + ' Downloading S2 images from ' + str(
        start_date) + ' to ' + str(end_date) + ' for each S2 tiles.')
for tile in tiles_list:
    print('')
    download_sentinel2(tile, start_date, end_date)
print(datetime.now().isoformat() + ' Successfully downloaded S2 images from ' + str(
    start_date) + ' to ' + str(end_date) + '.')

print(datetime.now().isoformat() + ' Unzipping S2 images downloaded.')
for file in os.listdir(cwd):
    file_path = os.path.join(cwd, file)
    if zipfile.is_zipfile(file_path):
        unzip(file_path, directory_images_s2)
print(datetime.now().isoformat() + ' Successfully unzipped S2 images.')

2021-11-08T11:24:44.868154 Downloading S2 images from 2020-10-01 to 2020-12-31 for each S2 tiles.









2021-11-08T13:28:58.523219 Successfully downloaded S2 images from 2020-10-01 to 2020-12-31.
2021-11-08T13:28:58.523401 Unzipping S2 images downloaded.
2021-11-08T13:29:21.804038Successfully extracted /home/wenger/Documents/doctorat_romain_wenger/create_dataset/SENTINEL2B_20201118-104739-093_L2A_T31UFP_D.zip
2021-11-08T13:29:40.338556Successfully extracted /home/wenger/Documents/doctorat_romain_wenger/create_dataset/SENTINEL2B_20201118-104732-754_L2A_T32ULU_D.zip
2021-11-08T13:30:03.249535Successfully extracted /home/wenger/Documents/doctorat_romain_wenger/create_dataset/SENTINEL2B_20201118-104724-709_L2A_T31UFQ_D.zip
2021-11-08T13:30:27.860423Successfully extracted /home/wenger/Documents/doctorat_romain_wenger/create_dataset/SENTINEL2B_20201105-103719-544_L2A_T32UMV_D.zip
2021-11-08T13:30:50.402832Successfully extracted /home/wenger/Documents/doctorat_romain_wenger/create_dataset/

### Ground reference

In [236]:
print(datetime.now().isoformat() + ' Rasterizing ground reference for each S2 tiles.')
if not os.path.exists(directory_ground_reference_raster):
    os.mkdir(directory_ground_reference_raster)

if not os.path.exists(directory_tmp):
    os.mkdir(directory_tmp)

dic_classes_urban = extract_dic(urban_regroup_classes.split(' '))
dic_classes_natural = extract_dic(natural_regroup_classes.split(' '))

attributes_merge = []
for key in dic_classes_urban:
    if dic_classes_urban[key] not in attributes_merge:
        attributes_merge.append(dic_classes_urban[key])

if -99 in attributes_merge:
    attributes_merge.remove(-99)

all_classes_urban = []
for key in dic_classes_urban:
    if key not in all_classes_urban:
        all_classes_urban.append(key)

all_classes_natural = []
for key in dic_classes_natural:
    if key not in all_classes_natural:
        all_classes_natural.append(key)

for tile in tiles_list:
    for img in os.listdir(directory_images_s2):
        if tile in img:
            path_band_ref = os.path.join(directory_images_s2, img, img + '_SRE_B3.tif')
            output_filename_tmp = os.path.join(cwd, directory_tmp,
                                                 tile + '_tmp.tif')
            shapefile_to_raster_tmp = os.path.join(cwd, directory_tmp,
                                                     path_shp_ground_reference.split('/')[2][:-4] + '_tmp.shp')
            output_filename_merge_tmp = os.path.join(cwd, directory_tmp,
                                                 tile + '_merge_tmp.tif')
            output_filename_merge_final = os.path.join(cwd, directory_ground_reference_raster,
                                                 tile + '_ground_reference.tif')

            # Changing ground reference projection and reclassify
            change_projection(path_band_ref, path_shp_ground_reference, attribute_shp_ground_reference_urban,
                              attribute_shp_ground_reference_natural, shapefile_to_raster_tmp, all_classes_urban, all_classes_natural)

            # Rasterizing shapefile
            rasterize_ground_ref(path_band_ref, shapefile_to_raster_tmp, attribute_shp_ground_reference_urban,
                                 output_filename_tmp)

            merge_classes(output_filename_tmp, merge_two_dicts(dic_classes_urban, dic_classes_natural), output_filename_merge_tmp)

            #Adding roads on reference data
            pixel_roads_value = 25
            computing_roads(os.path.join(cwd, directory_tmp), shapefile_roads, pixel_roads_value,
                            output_filename_merge_tmp, output_filename_merge_final)

            break

if os.path.exists(directory_tmp):
    shutil.rmtree(directory_tmp)

print(datetime.now().isoformat() + ' Successfully rasterized ground reference for each S2 tiles.')

2021-11-12T13:40:03.843206 Rasterizing ground reference for each S2 tiles.


AttributeError: 'module' object has no attribute 'rmdir'

### S1 (Doesn't work yet, use s1tiling : https://s1-tiling.pages.orfeo-toolbox.org/s1tiling/latest/install.html)

In [ ]:
print(datetime.now().isoformat() + ' Compress S1')
create_folders = False

if create_folders:
    for tile in os.listdir(directory_images_s1_not_compressed):
        os.mkdir(os.path.join(directory_images_s1_compressed, tile))
        for year in os.listdir(os.path.join(directory_images_s1_not_compressed, tile)):
            os.mkdir(os.path.join(directory_images_s1_compressed, tile, year))
            for month in os.listdir(os.path.join(directory_images_s1_not_compressed, tile, year)):
                os.mkdir(os.path.join(directory_images_s1_compressed, tile, year, month))
            

for tile in os.listdir(directory_images_s1_not_compressed):
    for year in os.listdir(os.path.join(directory_images_s1_not_compressed, tile)):
        for month in os.listdir(os.path.join(directory_images_s1_not_compressed, tile, year)):
            for img in os.listdir(os.path.join(directory_images_s1_not_compressed, tile, year, month)):
                command = 'gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" ' + os.path.join(directory_images_s1_not_compressed, tile, year, month, img) + ' ' +  os.path.join(directory_images_s1_compressed, tile, year, month, img)
                print(command)
                os.system(command)

print(datetime.now().isoformat() + ' Successfully compressed S1')

2021-11-15T14:47:32.133849 Compress S1
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/01/s1a_31TFN_vh_ASC_161_20200118t173159.tif /media/wenger/DATA2/S1_img/31TFN/2020/01/s1a_31TFN_vh_ASC_161_20200118t173159.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/01/s1a_31TFN_vh_ASC_161_20200130t173158.tif /media/wenger/DATA2/S1_img/31TFN/2020/01/s1a_31TFN_vh_ASC_161_20200130t173158.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/01/s1a_31TFN_vh_DES_037_20200110txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/01/s1a_31TFN_vh_DES_037_20200110txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/01/s1a_31TFN_vh_DES_037_20200122txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/01/s1a_31TFN_vh_DES_037_20200122txxxxxx.tif
gdal_translate -c

gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/02/s1b_31TFN_vv_ASC_161_20200229txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/02/s1b_31TFN_vv_ASC_161_20200229txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/02/s1b_31TFN_vv_DES_037_20200209txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/02/s1b_31TFN_vv_DES_037_20200209txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/02/s1b_31TFN_vv_DES_037_20200221txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/02/s1b_31TFN_vv_DES_037_20200221txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/03/s1a_31TFN_vh_ASC_161_20200306t173158.tif /media/wenger/DATA2/S1_img/31TFN/2020/03/s1a_31TFN_vh_ASC_161_20200306t173158.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co

gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/04/s1b_31TFN_vh_ASC_161_20200429txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/04/s1b_31TFN_vh_ASC_161_20200429txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/04/s1b_31TFN_vh_DES_037_20200409txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/04/s1b_31TFN_vh_DES_037_20200409txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/04/s1b_31TFN_vh_DES_037_20200421txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/04/s1b_31TFN_vh_DES_037_20200421txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co "ZSTD_LEVEL=8" /media/wenger/LaCie/S1_img/31TFN/2020/04/s1b_31TFN_vv_ASC_161_20200405txxxxxx.tif /media/wenger/DATA2/S1_img/31TFN/2020/04/s1b_31TFN_vv_ASC_161_20200405txxxxxx.tif
gdal_translate -co "COMPRESS=ZSTD" -co "PREDICTOR=3" -co

### Dataset creation

In [94]:
print(datetime.now().isoformat() + ' Calculating patches for each tile.')
'''if not os.path.exists(directory_dataset):
    os.mkdir(directory_dataset)

create_folders_dataset(directory_dataset)

for tile in tiles_list:
    print(datetime.now().isoformat() + ' Current tile is ' + str(tile))
    calculate_patches_tile(tile, directory_ground_reference_raster, directory_images_s2,
                           directory_images_s1,
                           directory_dataset, patch_size, method=1)'''

create_json_labels(directory_dataset)
print(datetime.now().isoformat() + ' Successfully calculated patches for each tile.')

2021-11-09T09:23:16.579565 Calculating patches for each tile.
2021-11-09T09:25:00.853008 Successfully calculated patches for each tile.


### Checking overlap

In [232]:
print(datetime.now().isoformat() + ' Checking overlapped patches.')
delete_zero_patch(directory_dataset)
delete_overlapping_patches(directory_dataset)
print(datetime.now().isoformat() + ' Successfully checked and deleted overlapped patches')

2021-11-09T16:49:43.038541 Checking overlapped patches.
2021-11-09T21:59:42.737621 Successfully checked and deleted overlapped patches


## Calculate stats

This part is dedicated to calculate some statistics over the dataset.

### Functions

In [100]:
def get_images_per_month(path_sat_img, sat):
    """
    Extract the number of image for the chosen satellite
    @params:
        path_dataset   	- Required  : path to the dataset
        sat             - Required  : satellite to create stats, 's1' or 's2'
    """
    assert (sat == 's1') or (sat == 's2')

    dic_res = {}

    if sat == 's1':
        for tile in os.listdir(path_sat_img):
            for year in os.listdir(os.path.join(path_sat_img, tile)):
                for month in os.listdir(os.path.join(path_sat_img, tile, year)):
                    path_img = os.path.join(path_sat_img, tile, year, month)
                    nb_file = len([name for name in os.listdir(path_img) if os.path.isfile(os.path.join(path_img, name))])
                    nb_file = nb_file/2

                    if month in dic_res.keys():
                        dic_res[int(month)] += nb_file
                    else:
                        dic_res[int(month)] = nb_file
    else:
        for folder_img in os.listdir(os.path.join(path_sat_img)):
            if 'SENTINEL' in folder_img:
                month = int(get_date_from_image_path(folder_img, s1=False)[4:6])

                if month in dic_res.keys():
                    dic_res[month] += 1
                else:
                    dic_res[month] = 1

    return dic_res

In [95]:
def get_number_pixel_gr(path_dataset):
    """
    Count the number of pixel per class in the dataset.
    @params:
        path_dataset   	- Required  : path to the dataset
    """
    dic_res = {}

    for patch in os.listdir(os.path.join(path_dataset, 'ground_reference')):
        dataset = gdal.Open(os.path.join(path_dataset, 'ground_reference', patch), GA_ReadOnly)
        array = dataset.GetRasterBand(1).ReadAsArray()

        unique, counts = np.unique(array, return_counts=True)
        dic_patch = dict(zip(unique, counts))

        if 0 not in array:
            for key in dic_patch.keys():
                if key in dic_res.keys():
                    dic_res[key] += dic_patch[key]
                else:
                    dic_res[key] = dic_patch[key]

    return dic_res

In [223]:
def make_bar_plot(dic, path_output, xlabel, ylabel, title, log=True):
    """
    Make bar plot with data contained in a dictionnary.
    @params:
        dic   	        - Required  : dictionnary with data
        path_output   	- Required  : output path of the png file
    """
    height = dic.values()
    bars = dic.keys()
    y_pos = np.arange(len(bars))
    
    plt.style.use('ggplot')
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = 'Helvetica'

    # Create bars
    plt.bar(y_pos, height, width= 0.7, align='center', alpha=0.4, color='blue', edgecolor = 'red')
    
    if log:
        plt.yscale("log") 
        # This is the location for the annotated text
        i = 1.0
        # Annotating the bar plot with the values (total death count)
        for i in range(len(bars)):
            y_stride = height[i] * 0.1
            plt.annotate("{:.2e}".format(height[i]), (-0.5 + i, height[i] + y_stride), size=4)
    else:
        i = 1.0
        # Annotating the bar plot with the values (total death count)
        for i in range(len(bars)):
            y_stride = height[i] * 0.02
            plt.annotate(height[i], (-0.1 + i, height[i] + y_stride), size=5)
    
    # Create names on the x-axis
    plt.xticks(y_pos, bars)
    
    plt.xlabel(str(xlabel), fontsize=9)
    plt.ylabel(str(ylabel), fontsize=9)
    plt.title(str(title), fontsize=9)

    plt.draw()
    plt.savefig(path_output, dpi=300)
    plt.clf()

In [225]:
output_stats = './stats_dataset'

if not os.path.exists(output_stats):
    os.mkdir(output_stats)
else:
    shutil.rmtree(output_stats)
    os.mkdir(output_stats)

path_dataset = './dataset'
path_s1 = '/media/wenger/LaCie/S1_img'
path_s2 = './S2_img'

dic_months_s1 = get_images_per_month(path_s1, sat='s1')
dic_months_s2 = get_images_per_month(path_s2, sat='s2')
dic_pixels = get_number_pixel_gr(path_dataset)

make_bar_plot(dic_months_s1, os.path.join(output_stats, 's1_bar_plot.png'), 'Months', 'Number of images', 'Number of images per month for Sentinel-1 time serie', log=False)
make_bar_plot(dic_months_s2, os.path.join(output_stats, 's2_bar_plot.png'), 'Months', 'Number of images', 'Number of images per month for Sentinel-2 time serie', log=False)
make_bar_plot(dic_pixels, os.path.join(output_stats, 'classes_bar_plot.png'), 'Classes', 'Number of pixels', 'Number of pixels per classes for ground reference')

<Figure size 432x288 with 0 Axes>